# Support Vector Machine (SVM) and Model Ensemble {-}

This assignment aims at familiarizing with training and testing Suppor Vector Machine classification model, along with exploiting the power of model ensemble technics.
- Load the data.
- Analyze the data.
- Remove outliers and clean the data.
- Use GridSearchCV to find the best set of SVM hyperparameters.
- Build, train and evaluate the SVM model.
- Separately build, train and evaluate the other four classifiers (Logistic regression, Naive Bayes, Decision Tree, Random Forest) on the same dataset, then compare their performance with the SVM model's.
- Apply three model ensemble technics, i.e., Bagging, Boosting and Stacking, to solve the problem, then compare their performance with each other and with the use of individual models. Draw conclusion from what has been observed.

The dataset you will be working on is 'data-breast-cancer.csv'. It is composed of attributes to build a prediction model.

**1. Load the data**

In [ ]:
# Load the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [ ]:
# Load the dataset
df = pd.read_csv("data-breast-cancer.csv")

In [ ]:
# Show some data samples
df.head()

,Unnamed: 0,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean
0,0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871
1,1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667
2,2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999
3,3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744
4,4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883


This is a dataset used to detect whether a patient has breast cancer depending on the following features:

- diagnosis: (label) the diagnosis of breast (label) tissues (M = malignant, B = benign).
- radius: distances from center to points on the perimeter.
- texture: standard deviation of gray-scale values.
- perimeter: perimeter of the tumor.
- area: area of the tumor.
- smoothness: local variation in radius lengths.
- compactness: is equal to (perimeter^2 / area - 1.0).
- concavity: severity of concave portions of the contour.
- concave points: number of concave portions of the contour.
- symmetry: symmetry of the tumor shape.
- fractal dimension: "coastline approximation" - 1.



**2. Analyze the data**

In [ ]:
# Drop "Unnamed:0" column as it does not contain useful information
df.drop(columns=["Unnamed: 0"], inplace=True)
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883


In [ ]:
# Shape of data frame
print("Data shape: " + str(df.shape) + "\n")

Data shape: (569, 11)



In [ ]:
# Show data information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   diagnosis               569 non-null    object 
 1   radius_mean             569 non-null    float64
 2   texture_mean            569 non-null    float64
 3   perimeter_mean          569 non-null    float64
 4   area_mean               569 non-null    float64
 5   smoothness_mean         569 non-null    float64
 6   compactness_mean        569 non-null    float64
 7   concavity_mean          569 non-null    float64
 8   concave points_mean     569 non-null    float64
 9   symmetry_mean           569 non-null    float64
 10  fractal_dimension_mean  569 non-null    float64
dtypes: float64(10), object(1)
memory usage: 49.0+ KB


In [ ]:
# Print out different types of diagnosis
df['diagnosis'].unique()

array(['M', 'B'], dtype=object)

In [ ]:
# Transform diagnosis to dummy variables using mapping
# Define mapping
mapping = {'M': 1, 'B': 0}

# Apply mapping to the DataFrame column
df['diagnosis'] = df['diagnosis'].map(mapping)

In [ ]:
# Drop duplicate samples
df = df.drop_duplicates(ignore_index=True)
df.shape


(569, 11)

In [ ]:
# Descibe the dataset
df.describe()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,0.372583,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798
std,0.483918,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060
min,0.000000,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960
25%,0.000000,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700
50%,0.000000,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540
75%,1.000000,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120
max,1.000000,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440


**3. Remove outliers and Clean the data**

In [ ]:
# Define function to remove outliers
def remove_outliers(df, columns):
    df_clean = df.copy()
    for col in columns:
        q = df[col].quantile(0.98)
        df_clean = df_clean[df_clean[col] < q]
    return df_clean

# Specify columns to remove outliers from
columns_to_clean = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
                    'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean']

# Call the function
df_clean = remove_outliers(df, columns_to_clean)

df_clean.shape

(512, 11)

In [ ]:
# Separate data features by removing the data label.
X = df_clean.drop(columns=["diagnosis"], axis=1)

# Assign data label to variable y
y = df_clean.diagnosis

# Split train/test with a random state
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, train_size=0.8)

In [ ]:
# Show some training samples
X_train.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean
199,14.45,20.22,94.49,642.7,0.09872,0.12060,0.11800,0.059800,0.1950,0.06466
284,12.89,15.70,84.08,516.6,0.07818,0.09580,0.11150,0.033900,0.1432,0.05935
198,19.18,22.49,127.50,1148.0,0.08523,0.14280,0.11140,0.067720,0.1767,0.05529
57,14.71,21.59,95.55,656.9,0.11370,0.13650,0.12930,0.081230,0.2027,0.06758
546,10.32,16.35,65.31,324.9,0.09434,0.04994,0.01012,0.005495,0.1885,0.06201


In [ ]:
# Check NAN values in the data
if df_clean.isna().any().any():
    print("There are missing values in the DataFrame.")
else:
    print("No missing values found in the DataFrame.")

No missing values found in the DataFrame.


**4. Use GridSearchCV to find the best set of SVM hyperparameters. Build, train and evaluate the SVM model.**

In [ ]:
# Initialize and use StandardScaler to normalize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_normalized_train = scaler.fit_transform(X_train)     # Fit and transform the training data
X_normalized_test = scaler.transform(X_test)           # Only transform the test data.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
param_grid = {"C": [0.01, 0.1, 1, 10, 100, 1000],
              "gamma": ["scale", 0.001, 0.005, 0.1]}
gridsearch = GridSearchCV(SVC(), param_grid, cv=10, scoring="f1", verbose=1)     # cv: number of folds in cross validation.

# Run grid search to find the best set of hyper-parameters
gridsearch.fit(X_normalized_train, y_train)


Fitting 10 folds for each of 24 candidates, totalling 240 fits


GridSearchCV(cv=10, estimator=SVC(),
             param_grid={'C': [0.01, 0.1, 1, 10, 100, 1000],
                         'gamma': ['scale', 0.001, 0.005, 0.1]},
             scoring='f1', verbose=1)

In [ ]:
# Best set of hyper-parameters
gridsearch.best_params_

{'C': 10, 'gamma': 'scale'}

In [ ]:
# Run SVM with the best set of hyper-parameters.
model = SVC(C=gridsearch.best_params_['C'], gamma=gridsearch.best_params_['gamma'])
model.fit(X_normalized_train, y_train)

SVC(C=10)

In [ ]:
# Evaluating the accuracy of the model
from sklearn.metrics import accuracy_score
accuracy_svm = accuracy_score(y_test, model.predict(X_normalized_test))
print("SVM Model Accuracy:", accuracy_svm)

# Show evaluation metrics on the test set
from sklearn.metrics import classification_report
print(classification_report(y_test, model.predict(X_normalized_test)))

SVM Model Accuracy: 0.941747572815534
              precision    recall  f1-score   support

           0       0.94      0.97      0.96        67
           1       0.94      0.89      0.91        36

    accuracy                           0.94       103
   macro avg       0.94      0.93      0.94       103
weighted avg       0.94      0.94      0.94       103



**5. Separately build, train and evaluate the other four classifiers (Logistic regression, Naive Bayes, Decision Tree, Random Forest) on the same dataset, then compare their performance with the SVM model's.**

**5.1. Logistic regression**

In [ ]:
# Load the libraries
from sklearn.linear_model import LogisticRegression

grid_search={"C":[0.01, 0.1, 1]} # Define the values of hyperparameter C we want to try
logmodel=LogisticRegression() # Initialize the logistic regression model
logmodel_cv=GridSearchCV(logmodel, grid_search, cv=5) # Set up GridSearchCV to find the best value of hyperparameter C, with 5-fold cross validation, i.e., cv=5.
logmodel_cv.fit(X_normalized_train, y_train) # Train the model using GridSearchCV

logmodel = LogisticRegression(C=logmodel_cv.best_params_['C'])  # Initialize Logistic Regression model with the best value of hyper parameter C
logmodel.fit(X_normalized_train, y_train)       # Train the model

# Make prediction on the test data
pred_y = logmodel.predict(X_normalized_test)

# Evaluating the accuracy of the model
accuracy_lr = accuracy_score(y_test, pred_y)
print("Logistic Regression Model Accuracy:", accuracy_lr)

# Show evaluation metrics on the test set
print(classification_report(y_test, pred_y))

Logistic Regression Model Accuracy: 0.8932038834951457
              precision    recall  f1-score   support

           0       0.88      0.97      0.92        67
           1       0.93      0.75      0.83        36

    accuracy                           0.89       103
   macro avg       0.90      0.86      0.88       103
weighted avg       0.90      0.89      0.89       103



**5.2. Naive Bayes**

In [ ]:
# Load the libraries
from sklearn.naive_bayes import GaussianNB    # Initialize Gaussian Naive Bayes model
naive_model = GaussianNB()

# Define the values of hyperparameter var_smoothing we want to try
grid_search={"var_smoothing":[1e-2, 1e-3, 1e-4, 1e-5,]}

# Set up GridSearchCV to find the best value of hyperparameter var_smoothing, with 5-fold cross validation
naive_cv=GridSearchCV(naive_model, grid_search, cv=5)

# Train the model using GridSearchCV
naive_cv.fit(X_normalized_train, y_train)

GridSearchCV(cv=5, estimator=GaussianNB(),
             param_grid={'var_smoothing': [0.01, 0.001, 0.0001, 1e-05]})

In [ ]:
# Initialize Gaussian Naive Bayes model with the best value of hyperparameter var_smoothing
naive_normal = GaussianNB(var_smoothing=naive_cv.best_params_['var_smoothing'])

# Train the model
naive_normal.fit(X_normalized_train, y_train)

# Make prediction on the test data
pred_y = naive_normal.predict(X_normalized_test)

# Evaluating the accuracy of the model
accuracy_nb = accuracy_score(y_test, pred_y)
print("Naive Bayes Model Accuracy:", accuracy_nb)

# Show evaluation metrics on the test set
print(classification_report(y_test, pred_y))


Naive Bayes Model Accuracy: 0.883495145631068
              precision    recall  f1-score   support

           0       0.91      0.91      0.91        67
           1       0.83      0.83      0.83        36

    accuracy                           0.88       103
   macro avg       0.87      0.87      0.87       103
weighted avg       0.88      0.88      0.88       103



**5.3. Decision Tree**

In [ ]:
# Check if the data is imbalanced then apply SMOTE if neccessary

# Before oversampling
print("Before oversampling: " + str(X_train.shape))
print(np.unique(y_train, return_counts=True))                  # Print number of labels, label '1' dominates '0'

Before oversampling: (409, 10)
(array([0, 1]), array([271, 138]))


In [ ]:
# Apply oversampling method for label '1'
from imblearn.over_sampling import SMOTE     # Load the SMOTE library
smote = SMOTE(random_state=5)                # Initialize SMOTE
X_train_oversampling, y_train_oversampling = smote.fit_resample(X_train, y_train)     # Oversample label '1' (minority class) in the training set

In [ ]:
# After oversampling
print("After oversampling: " + str(X_train_oversampling.shape))
print(np.unique(y_train_oversampling, return_counts=True))     # Print number of labels, now label '0' and '1' have the same number of labels.

After oversampling: (542, 10)
(array([0, 1]), array([271, 271]))


In [ ]:
# Import GridSearchCV for finding the best hyper-parameter set.
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

params = {"criterion": ["gini", "entropy"],             # Criterion to evaluate the purity.
         "max_depth": [3, 5],                           # Maximum depth of the tree
         "min_samples_split": [4, 8]}                   # Stop splitting condition.

grid_search = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=params, cv=5)

In [ ]:
# Run the search on oversampled training data samples.
grid_search.fit(X_train_oversampling, y_train_oversampling)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'], 'max_depth': [3, 5],
                         'min_samples_split': [4, 8]})

In [ ]:
# Build a decision tree model pipeline from the best set of hyper-parameters found
model_dt = DecisionTreeClassifier(criterion=grid_search.best_params_['criterion'], max_depth=grid_search.best_params_['max_depth'], min_samples_split=grid_search.best_params_['min_samples_split'])

In [ ]:
# Train the decision tree model
model_dt.fit(X_train_oversampling, y_train_oversampling)

DecisionTreeClassifier(max_depth=3, min_samples_split=8)

In [ ]:
# Make prediction on the original test set (after training on the over-sampled training set).
pred_y = model_dt.predict(X_test)

# Evaluating the accuracy of the model
accuracy_dt = accuracy_score(y_test, pred_y)
print("Decision Tree Model Accuracy:", accuracy_dt)

# Show evaluation metrics on the test set
print(classification_report(y_test, pred_y))

Decision Tree Model Accuracy: 0.9223300970873787
              precision    recall  f1-score   support

           0       0.95      0.93      0.94        67
           1       0.87      0.92      0.89        36

    accuracy                           0.92       103
   macro avg       0.91      0.92      0.92       103
weighted avg       0.92      0.92      0.92       103



**5.4. Random forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

params = {"criterion": ["gini", "entropy"],             # Criterion to evaluate the purity.
         "max_depth": [7, 9, 11],                           # Maximum depth of the tree
         "min_samples_split": [8, 12, 16]}                   # Stop splitting condition.

grid_search_rf = GridSearchCV(estimator=RandomForestClassifier(n_estimators=10, n_jobs=10), param_grid=params, cv= 5) # Number of trees in the forest is 10

# Run the search on oversampled training data samples.
grid_search_rf.fit(X_train_oversampling, y_train_oversampling)     # Train the RandomForest

GridSearchCV(cv=5, estimator=RandomForestClassifier(n_estimators=10, n_jobs=10),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [7, 9, 11],
                         'min_samples_split': [8, 12, 16]})

In [ ]:
# Build a Random Forest model pipeline from the best set of hyper-parameters found
model_rf = RandomForestClassifier(n_estimators=10, random_state=1, criterion=grid_search_rf.best_params_['criterion'], max_depth=grid_search_rf.best_params_['max_depth'], min_samples_split=grid_search_rf.best_params_['min_samples_split'])     # Initialize the RandomForest

In [ ]:
# Train the Random Forest model
model_rf.fit(X_train_oversampling, y_train_oversampling)

RandomForestClassifier(criterion='entropy', max_depth=9, min_samples_split=8,
                       n_estimators=10, random_state=1)

In [ ]:
# Make prediction on the original test set (after training on the over-sampled training set).
pred_y = model_rf.predict(X_test)

# Evaluating the accuracy of the model
accuracy_rf = accuracy_score(y_test, pred_y)
print("Random Forest Model Accuracy:", accuracy_rf)

# Show evaluation metrics on the test set
print(classification_report(y_test, pred_y))

Random Forest Model Accuracy: 0.9223300970873787
              precision    recall  f1-score   support

           0       0.97      0.91      0.94        67
           1       0.85      0.94      0.89        36

    accuracy                           0.92       103
   macro avg       0.91      0.93      0.92       103
weighted avg       0.93      0.92      0.92       103



**5.5. Comparision**


*   In term of accuracy, SVM performs the best with very high accuracy value of 0.94. Second performance standing are Decision Tree and Random Forest (0.92). The two last models are Logistic Regression (0.89) and Naive Baynes (0.88).



**6. Apply three model ensemble technics, i.e., Bagging, Boosting and Stacking, to solve the problem, then compare their performance with each other and with the use of individual models. Draw conclusion from what has been observed.**

In [ ]:
# Load the libraries
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

**6.1. Bagging with Support Vector Machine (SVM)**

In [ ]:
# Creating a Support Vector Machine Classifier as the base estimator
base_svm = SVC(kernel='linear', C=1.0)

# Create a Bagging Classifier with SVM as the base model
bagging_clf = BaggingClassifier(estimator=base_svm, n_estimators=10, max_samples=0.5)

# Training the Bagging Classifier
bagging_clf.fit(X_train, y_train)

# Making predictions on the test set
pred_y = bagging_clf.predict(X_test)

# Evaluating the accuracy
accuracy_bagging = accuracy_score(y_test, pred_y)
print("Bagging Accuracy:", accuracy_bagging)

# Show evaluation metrics on the test set
print(classification_report(y_test, pred_y))

Bagging Accuracy: 0.883495145631068
              precision    recall  f1-score   support

           0       0.90      0.93      0.91        67
           1       0.85      0.81      0.83        36

    accuracy                           0.88       103
   macro avg       0.88      0.87      0.87       103
weighted avg       0.88      0.88      0.88       103



**6.2. Boosting with AdaBoost, Gradient Boosting and XGBoost**

In [ ]:
# AdaBoost

# Create an AdaBoost Classifier with Decision Tree as the base model
ada_clf = AdaBoostClassifier(estimator = DecisionTreeClassifier(), n_estimators=10)

# Train the AdaBoost Classifier
ada_clf.fit(X_train, y_train)

# Making predictions on the test set
pred_y_ada = ada_clf.predict(X_test)

# Evaluating the accuracy of the model
accuracy_ada = accuracy_score(y_test, pred_y_ada)
print("AdaBoost Accuracy:", accuracy_ada)

# Show evaluation metrics on the test set
print(classification_report(y_test, pred_y_ada))

AdaBoost Accuracy: 0.883495145631068
              precision    recall  f1-score   support

           0       0.97      0.85      0.90        67
           1       0.77      0.94      0.85        36

    accuracy                           0.88       103
   macro avg       0.87      0.90      0.88       103
weighted avg       0.90      0.88      0.89       103



In [ ]:
# Gradient Boosting

# Create a Gradient Boosting Classifier which uses Decision Tree as boosting model by default
gb_clf = GradientBoostingClassifier(n_estimators=10, learning_rate=0.1)

# Train the Gradient Boosting Classifier
gb_clf.fit(X_train, y_train)

# Making predictions on the test set
pred_y_gb = gb_clf.predict(X_test)

# Evaluating the accuracy of the model
accuracy_gb = accuracy_score(y_test, pred_y_gb)
print("Gradient Boosting Accuracy:", accuracy_gb)

# Show evaluation metrics on the test set
print(classification_report(y_test, pred_y_gb))

Gradient Boosting Accuracy: 0.912621359223301
              precision    recall  f1-score   support

           0       0.95      0.91      0.93        67
           1       0.85      0.92      0.88        36

    accuracy                           0.91       103
   macro avg       0.90      0.91      0.91       103
weighted avg       0.92      0.91      0.91       103



In [ ]:
# XGBoost

# Create an XGBoost Classifier
xgb_clf = XGBClassifier(n_estimators=100, learning_rate=0.1)

# Train the XGBoost Classifier
xgb_clf.fit(X_train, y_train)

# Making predictions on the test set
pred_y_xgb = xgb_clf.predict(X_test)

# Evaluating the accuracy of the model
accuracy_xgb = accuracy_score(y_test, pred_y_xgb)
print("XGBoost Accuracy:", accuracy_xgb)

# Show evaluation metrics on the test set
print(classification_report(y_test, pred_y_xgb))

XGBoost Accuracy: 0.9320388349514563
              precision    recall  f1-score   support

           0       0.97      0.93      0.95        67
           1       0.87      0.94      0.91        36

    accuracy                           0.93       103
   macro avg       0.92      0.93      0.93       103
weighted avg       0.93      0.93      0.93       103



XGBoost performs the best in all evaluation metrics of accuracy, precision, recall and f1-score.

**6.3. Stacking**

In [ ]:
# K-Nearest Neighbor (KNN)
# Train a K-Nearest Neighbor (KNN) model
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()                           # Initialize KNN model.
params_knn = {'n_neighbors': np.arange(1, 25)}         # n_neighbors in KNeighborsClassifier() indicates the number of neighbors K.
knn_gs = GridSearchCV(knn, params_knn, cv=5)           # Initialize GridSearchCV to find an optimal value of K.
knn_gs.fit(X_train, y_train)                           # Fit GridSearch in training set to find the optimal K.

# Best number of neighbors K
knn_best = knn_gs.best_estimator_
print(knn_gs.best_params_)

{'n_neighbors': 4}


In [ ]:
# Support Vector Machine (SVM)
# Train a Support Vector Machine (SVM) model
svm = SVC()
params_svm = {"C": [0.1, 1, 10, 100]}

svm_gs = GridSearchCV(svm, params_svm, cv=5)    # Initialize GridSearchCV to find an optimal value if the hyperparameter C.
svm_gs.fit(X_train, y_train)                    # Fit GridSearch in training set to find the optimal C.

# Best value of the hyperparameter C.
svm_best = svm_gs.best_estimator_
print(svm_gs.best_params_)

{'C': 10}


In [ ]:
# Random Forest
# Train a Random Forest classifier
rf = RandomForestClassifier()                        # Initialize a Random Forest Classifier.
params_rf = {'n_estimators': [50, 100, 200]}         # n_estimator in RandomForestClassifier(...) indicates the number of Trees in the Forest.
rf_gs = GridSearchCV(rf, params_rf, cv=5)            # Initialize GridSearchCV to find an optimal number of Trees.
rf_gs.fit(X_train, y_train)                          # Fit GridSearch in training set to find the optimal number of Trees.

# Best number of Trees.
rf_best = rf_gs.best_estimator_
print(rf_gs.best_params_)

{'n_estimators': 100}


In [ ]:
# Logistic Regression
# Train a Logistic Regression model
log_reg = LogisticRegression(solver='lbfgs', max_iter=1000)   # Initialize Logistic Regression model.
log_reg.fit(X_train, y_train)                                 # Fit the model to training set.

LogisticRegression(max_iter=1000)

In [ ]:
# Model Testing
# Print accuracy of single models on the test set
print('KNN: {}'.format(knn_best.score(X_test, y_test)))                     # KNN accuracy
print('SVM: {}'.format(svm_best.score(X_test, y_test)))                     # SVM accuracy
print('Random Forest: {}'.format(rf_best.score(X_test, y_test)))            # Random Forest accuracy
print('Logistic Regression: {}'.format(log_reg.score(X_test, y_test)))      # Logistic Regression accuracy

KNN: 0.8932038834951457
SVM: 0.8640776699029126
Random Forest: 0.9029126213592233
Logistic Regression: 0.883495145631068


In [ ]:
# Model ensembling

from sklearn.ensemble import VotingClassifier
# Ensemble the four models using hard (majority) voting
estimators=[('knn', knn_best), ('svm', svm_best), ('rf', rf_best), ('log_reg', log_reg)]    # Initialize base models in the ensemble
ensemble = VotingClassifier(estimators, voting='hard')                                      # Define how to ensemble them, i.e., hard voting

# Train the model ensemble on the training set
ensemble.fit(X_train, y_train)          # Train the ensemble on the training set
ensemble.score(X_test, y_test)          # Test the ensemble on the test set

0.883495145631068

Boosting with XGBoost performs the best compared to Bagging and Stacking with accuracy of 0.93. However, this still performs worsen than SVM model individually with accuracy of 0.94. This contradicts with theoritical conclusion of model ensemble can achieve higher accuracy than single model. This might be due to some of factors such as model diversity, overfitting and data quality which can lead to lower model accuracy.